In [33]:
from urllib.request import urlopen, urlretrieve
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import time
import requests 
from fake_headers import Headers 
from difflib import get_close_matches

df = pd.DataFrame()

In [34]:
sites = {'link': [], 'title': []}

page = 'https://www.banki.ru/banks/?PAGEN_1={}#results'

# resp = urlopen(pages[0]) # скачиваем файл
# html = resp.read().decode('utf8') # считываем содержимое
# soup = BeautifulSoup(html, 'html.parser') # делаем суп


for i in range(1, 9):
    search_page = page.format(i)
    resp = urlopen(search_page) # скачиваем файл
    html = resp.read().decode('utf8') # считываем содержимое
    time.sleep(1)
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    for td in soup.find_all('td'):
        if td.find('a'):
            if '/banks/bank/' in td.a.get('href'):
                sites['link'].append('https://www.banki.ru' + td.a.get('href'))
                sites['title'].append(td.a.get('title'))
    time.sleep(2)            
        

In [35]:
df_sites = pd.DataFrame(sites)

In [36]:
df_sites

link           title
0          https://www.banki.ru/banks/bank/talkbank/        TalkBank
1       https://www.banki.ru/banks/bank/absolutbank/    Абсолют Банк
2          https://www.banki.ru/banks/bank/avangard/        Авангард
3         https://www.banki.ru/banks/bank/aversbank/           Аверс
4      https://www.banki.ru/banks/bank/avtogradbank/    Автоградбанк
..                                               ...             ...
360        https://www.banki.ru/banks/bank/ubs-bank/    Ю Би Эс Банк
361  https://www.banki.ru/banks/bank/yug-investbank/   Юг-Инвестбанк
362   https://www.banki.ru/banks/bank/unicreditbank/  ЮниКредит Банк
363       https://www.banki.ru/banks/bank/unistream/        Юнистрим
364    https://www.banki.ru/banks/bank/yarinterbank/     Яринтербанк

[365 rows x 2 columns]

In [54]:
pattern = r'[^\w\d,]'
pattern2 = r'\s+'




peoples = {'руководитель': [], 'место': [], 'наименование банка': [], 'сайт': []}
for i, link in enumerate(df_sites['link'].to_list()):
    #print(i)
    peoples['руководитель'].append('Не найдено')
    peoples['место'].append('не найдено')
    peoples['наименование банка'].append('не найдено')
    peoples['сайт'].append('не найдено')
    time.sleep(0.25)
    resp = urlopen(link) # скачиваем файл
    html = resp.read().decode('utf8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    for dt in soup.find_all('dt'):
        if 'руководитель' in dt.text.lower():
            try:
                peoples['руководитель'][i] = re.sub(pattern2, ' ', re.sub(pattern, ' ', dt.findNext('dd').text.strip()))
            except:
                peoples['руководитель'][i] = 'Не найдено'
        if 'нетто' in dt.text.lower():
            try:
                peoples['место'][i] = re.sub(pattern2, ' ', re.sub(pattern, ' ', dt.findNext('dd').text.strip()))
            except:
                peoples['место'][i] = 'Не найдено'
        if 'официальный сайт' in dt.text.lower():
            try:
                peoples['сайт'][i] = dt.findNext('dd').text.strip()
            except:
                peoples['сайт'][i] = 'Не найдено'        
    for strong in soup.find_all('strong'):   
        if strong.has_attr('class') and "font-size-x-large" in strong.get('class'):
            try:
                peoples['наименование банка'][i] = strong.text.strip()
            except:
                peoples['наименование банка'][i] = 'Не найдено'

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


In [41]:
df_all = pd.concat([df_sites, pd.DataFrame(peoples)], axis=1)

In [42]:
df_all.to_excel('Банки_ру_руководители.xlsx')

In [ ]:
# дубликаты бывают в банках БКС там например

## Обработаем файл

In [85]:
df_all = pd.read_excel('Банки_ру_руководители.xlsx')

In [86]:
spisok = pd.read_excel(r'D:\FTP-DATA\Рассылка_шаблон\Список.xlsx')

In [87]:
df_all_list = df_all['наименование банка'].str.lower().to_list()


In [88]:
spisok_list = spisok['Банк_кратко'].str.lower().to_list()

In [89]:
dct = {}
for element in df_all_list:
    dct.setdefault(element, [])
    dct[element] = ''.join(get_close_matches(element, spisok_list, n=1, cutoff=0.85))

In [90]:
df_all['Совпадения_банков'] = dct.values()

In [91]:
df_all.to_excel('Проверка.xlsx')

In [93]:
spisok['кратко'] = spisok['Банк_кратко'].str.lower().to_list()

In [97]:
df_all = df_all.merge(spisok[['Банк_кратко', 'кратко']], how='left', left_on='Совпадения_банков', right_on='кратко')

In [101]:
df_all.to_excel('Проверка.xlsx')

In [ ]:
# вручную проверил совпадения

In [102]:
prov = pd.read_excel('Проверка.xlsx')

In [104]:
prov.drop('Банк_кратко', axis=1, inplace=True)

In [106]:
prov['Руководитель_ФИО'] = prov['руководитель'].apply(lambda x: x[:x.find(' по информации')])
prov['Актуальность_информации'] = prov['руководитель'].apply(lambda x: x[x.find('по информации'):])


In [111]:
def fio_kratko(x):
    x = x.split()
    try:
        return f'{x[0]} {"".join([i[0] + "." for i in x[1:]])}'
    except:
        return x

prov['ФИО_кратко'] = prov['Руководитель_ФИО'].apply(fio_kratko)

In [136]:
pattern = r'\s+((\d{1,})\s{1,}место по России)'

def place(x):
    
    try:
        return re.search(pattern, x).group(2)
    except:
        return x



prov['Место в рейтинге'] = prov['место'].apply(place)

In [138]:
prov.to_excel('Список_2.xlsx')